# Functions from the previous assignments


## Modulation and demodulation

In [358]:
import numpy as np

def qpsk_modulation(bits):
    qpsk_constellation=np.array([complex(1,1), complex(-1,1), complex(-1,-1), complex(1,-1)])

    try:
        return qpsk_constellation[bits]
    except IndexError:
        raise ValueError('{} is to large for this constellation.'.format(bin(bits)))        


def qpsk_demodulation(received_symbol):

    if received_symbol.real >= 0:
        if received_symbol.imag >=0:            
            return 0
        else:
            return 3
    else:
        if received_symbol.imag >=0:            
            return 1
        else:
            return 2


## Coding and decoding

In [359]:
def encode(u):   
    G = np.matrix('1, 0, 0, 0, 1, 1, 0;0,1,0,0,1,0,1;0 0 1 0 0 1 1;0 0 0 1 1 1 1')
    x=u*G
    x=x%2 
    return x

def decode(x_hat):
    H = np.matrix('1,1,0,1,1,0,0;1,0,1,1,0,1,0;0,1,1,1,0,0,1')
    s=x_hat*H.transpose()
    s=s%2
     
    if s[0,0]==0 and s[0,1]==0 and s[0,2]==0:
        return [x_hat[0],x_hat[1],x_hat[2],x_hat[3]]
    else:
        for i in range(7):
            if s[0,0]==H.transpose()[i,0] and s[0,1]==H.transpose()[i,1] and s[0,2]==H.transpose()[i,2]:
                break
        x_hat[i]=x_hat[i]+1
        x_hat[:]=[y%2 for y in x_hat]
        return [x_hat[0],x_hat[1],x_hat[2],x_hat[3]]

## Interleaving and deinterleaving

In [360]:
def interleave(A):
    return np.transpose(A)

def deinterleave(A):
    return np.transpose(A)

# Addition of noise

I have chosen the value for the variable scale on the function awgn_channel that gave me reasonable answers at the end. If the noise is too big, the use of coding and interleaving appear to be not efficient when comparing to the case without these procedures.

In [361]:
def awgn_channel(symbols):
     noise = np.random.normal(loc=0, scale=0.5, size=(1, 2)).view(np.complex128)
     return  symbols + noise

#print check
print(awgn_channel(-1+1j))

[[-0.87580667+1.81523213j]]


# The System

For the final code I had to adapt the "interface" between one function and the other, as I didn't take that into account in the previous assignments. That took me a lot of thinking and time, that I thought wouldn't be so important to explain in details... The blocks are highlighted as comments in the python code.

In [362]:
length=7*2**12
data = np.random.randint(0, 2, length)
data2 = np.zeros((int(length/4),4))
y=0
for x in range(len(data)):
    data2[y,x%4]=data[x]
    if (x+1)%4==0:
        y=y+1
        
# ENCODING
encoded = np.zeros((len(data2),7))
for x in range(len(data2)):
    encoded[x,:]=encode(data2[x,:])
    
# INTERLEAVING
interleaved = np.zeros((len(data2),7))
x=0
for y in range(len(encoded)):
    if (y+1)%7==0:
        interleaved[x:y+1,0:7]=interleave(encoded[x:y+1,0:7])
        x=x+7
        
# MODULATION
row_vector = np.zeros(7*len(interleaved))
for x in range(len(row_vector)):
    row_vector[x]=interleaved[int(x/7),x%7]

decimal_vector = np.zeros(int(len(row_vector)/2))
y=0
for x in range(len(row_vector)):
    if (x+1)%2==0:
        a=row_vector[x-1]
        b=row_vector[x]
        decimal_vector[int(x/2)]=a*2+b

modulated = np.random.normal(loc=0, scale=0, size=(len(decimal_vector),2)).view(np.complex128)

for x in range(len(decimal_vector)):
    modulated[x]=qpsk_modulation(int(decimal_vector[x]))
    
    
# ADD THE NOISE
received = np.random.normal(loc=0, scale=0, size=(len(decimal_vector),2)).view(np.complex128)
for x in range(len(decimal_vector)):
    received[x]=awgn_channel(modulated[x])
    
#DEMODULATION
demodulated=np.zeros(len(decimal_vector))
for x in range(len(decimal_vector)):
    demodulated[x]=qpsk_demodulation(received[x])
    
#DEINTERLEAVING
row_vector2=np.zeros(len(row_vector))
for x in range(len(decimal_vector)):
    row_vector2[2*x]=int(demodulated[x]/2)
    row_vector2[2*x+1]=demodulated[x]%2
    
received_matrix = np.zeros((len(data2),7))
for x in range(len(row_vector2)):
    received_matrix[int(x/7),x%7]=row_vector2[x]
    
deinterleaved = np.zeros((len(data2),7))
x=0
for y in range(len(deinterleaved)):
    if (y+1)%7==0:
        deinterleaved[x:y+1,0:7]=deinterleave(received_matrix[x:y+1,0:7])
        x=x+7

#DECODING
decoded=np.zeros((len(deinterleaved),4))
for x in range(len(decoded)):
    decoded[x,:]=decode(deinterleaved[x,:])
    
#FINAL VECTOR
y=0
data_hat=np.zeros(len(data))
for x in range(len(data)):
    data_hat[x]=decoded[y,x%4]
    if (x+1)%4==0:
        y=y+1
        
#COMPARISION
ber=1-np.sum(data == data_hat)/length
print(ber*100)

0.8963448660714302


In [363]:
#WITHOUT CODING AND INTERLEAVING

decimal_vector2 = np.zeros(int(len(data)/2))
y=0
for x in range(len(data)):
    if (x+1)%2==0:
        a=data[x-1]
        b=data[x]
        decimal_vector2[int(x/2)]=a*2+b

#MODULATION
modulated2 = np.random.normal(loc=0, scale=0, size=(len(decimal_vector2),2)).view(np.complex128)

for x in range(len(decimal_vector2)):
    modulated2[x]=qpsk_modulation(int(decimal_vector2[x]))
    
#ADD NOISE
received2 = np.random.normal(loc=0, scale=0, size=(len(decimal_vector2),2)).view(np.complex128)
for x in range(len(decimal_vector2)):
    received2[x]=awgn_channel(modulated2[x])
    
#DEMODULATION
demodulated2=np.zeros(len(decimal_vector2))
for x in range(len(decimal_vector2)):
    demodulated2[x]=qpsk_demodulation(received2[x])
    
row_vector3=np.zeros(len(data))
for x in range(len(demodulated2)):
    row_vector3[2*x]=int(demodulated2[x]/2)
    row_vector3[2*x+1]=demodulated2[x]%2
    
#COMPARISION
row_vector3[:]=[int(y) for y in row_vector3]
ber2=1-np.sum(data == row_vector3)/length
print(ber2*100)

3.50864955357143


The bit error rates (in percentage) are displayed for the case with and without coding and interleaving, respectively. Comparing the values it is clear that the coding and interleaving procedures are important to improve the reliability of our system.